In [254]:
import pandas as pd
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import os
dir_path = os.getcwd()


In [255]:
d_types = ['data', 'test']
data = dict()
for d_type in d_types:
        path = dir_path +"/"+"classification_data_HWK2/EMGaussian." + d_type 
        data[d_type] = pd.read_csv(path, sep=' ', header=None).values

In [272]:
K = 4
T = len(Y_t_data) - 1

In [273]:
# We use the parameters found in HW2 - Q3 (We use our data)
pi = [0.20, 0.24, 0.25, 0.31]
centers = [[3.79, -3.64], [ 3.99, -3.64 ], [ -2.03, 4.16 ], [ -3.08, -3.56 ]]

sigma_0 = np.matrix([[ 0.87, 0.06 ], [ 0.06, 2.21]])
sigma_1 = np.matrix([[ 0.20, 0.22 ], [ 0.22, 10.40]])
sigma_2 = np.matrix([[ 2.92, 0.17 ], [ 0.17, 2.77]])
sigma_3 = np.matrix([[ 6.14, 5.94 ], [ 5.94, 6.07]])

sigmas = [sigma_0, sigma_1, sigma_2, sigma_3]

multivar = dict()
for k in range(K):
    m = scipy.stats.multivariate_normal(centers[k], sigmas[k])
    multivar[k] =m
    
# We also need to initiate A as it is not specified what is its value
# We follow Mr Chopin's advice to have strong weight on the diagonal
A =  1/10 * np.ones((4,4)) + 6/10 * np.identity(4)

log_A = np.log(A)

Y_t_data = data['data']

In [274]:
def computeAlpha(z, Y_t_data, A,  t): 
    if t == 0:
        return pi[z]
    p_yt_zt = multivar[z].pdf(Y_t_data[t])
    alpha = 0
    for i in range(4):
        alpha += p_yt_zt * A[z, i] * computeAlpha(i, Y_t_data, A, t-1)
        
    return alpha

In [275]:
print(computeAlpha(2, Y_t_data, A, 8))
print(np.log(computeAlpha(2, Y_t_data, A, 8)))

9.711462643780658e-22
-48.38356514218259


In [317]:
def computeBeta(z, Y_t_data, A,  t): 
    if t == T:
        return 1
    beta = 0
    for i in range(4):
        beta += A[z, i] * multivar[i].pdf(Y_t_data[t]) *  computeBeta(i, Y_t_data, A, t+1)
    return beta

In [318]:
print(computeBeta(2, Y_t_data, A, T-3))
print(np.log(computeBeta(2, Y_t_data, A, T-3)))

4.810772938040479e-07
-14.547237885771235


In [319]:
# we note that values are very low, we thus need to use the log version to prevent numerical errors
log_A = np.log(A)
log_pi = np.log(pi)

# We will also use a matrix to store the already computed values of logAlpha and logBeta to prevent us from recomputing
# the whole recursivity at every step


In [320]:
def computeLogAlpha(z, Y_t_data, log_A, log_pi, memory_log_alpha, multivar,  t): 
    if t == 0:
        memory_log_alpha[z, t] = log_pi[z]
        return log_pi[z]
    p_yt_zt = multivar[z].pdf(Y_t_data[t])
    log_alpha = 0
    log_inside_sum = [0,0,0,0]
    for i in range(4):
        log_alpha_i_tminus1 = 0 
        if memory_log_alpha[i, t-1] == 0:
            log_alpha_i_tminus1 = computeLogAlpha(i, Y_t_data, log_A, log_pi, memory_log_alpha,multivar,  t-1)
        else:
            log_alpha_i_tminus1 = memory_log_alpha[i, t-1]
        log_inside_sum[i] = log_alpha_i_tminus1 + log_A[z, i]
        
    max_log = np.max(log_inside_sum)
    
    log_alpha += np.log(p_yt_zt) + max_log + np.log(np.sum(np.exp(log_inside_sum - max_log)))
    
    memory_log_alpha[z, t] = log_alpha  
    return log_alpha

In [321]:
memory_log_alpha = np.zeros((4, T + 1))
computeLogAlpha(2, Y_t_data, log_A, log_pi, memory_log_alpha,multivar,  T)

-2624.8874882902496

In [324]:
def computeLogBeta(z, Y_t_data, log_A, memory_log_beta, multivar,  t): 
    if t == T:  
        memory_log_beta[z, t] = 0
        return 0
    log_inside_sum = [0,0,0,0]
    for i in range(4):
        log_beta_i_tplus1 = 0 
        if memory_log_beta[i, t+1] == 0:
            log_beta_i_tplus1 = computeLogBeta(i, Y_t_data, log_A, memory_log_beta, multivar, t+1)
        else:
            log_beta_i_tplus1 = memory_log_beta[i, t+1]
        log_inside_sum[i] = log_A[z, i] + np.log(multivar[i].pdf(Y_t_data[t])) +  log_beta_i_tplus1
    
    max_log = np.max(log_inside_sum)
    log_beta = max_log + np.log(np.sum(np.exp(log_inside_sum - max_log)))  
    memory_log_beta[z, t] = log_beta  
    return log_beta

In [326]:
memory_log_beta = np.zeros((4, T + 1))
print(computeLogBeta(2, Y_t_data, log_A, memory_log_beta, multivar, T-3))

-14.547237885771237


## 3. EM Algorithm

We will use the formulas found computed on the pdf file 
We will also initialize $\Pi_0$, $\mu_i$ and $\Sigma_i$ with the values found in the previous homework
For A, we will follow Mr Chopin's advice and we'll initiate our matrix with strong diagonal weights

In [412]:
def computeSmoothing(memory_log_alpha, memory_log_beta):
    p_zt_i = np.zeros((4, T+1))
    for t in range(T+1):
        memory_log_alpha_t = memory_log_alpha[:, t]
        memory_log_beta_t = memory_log_alpha[:, t]
        sum_memory_log_t = memory_log_alpha_t + memory_log_beta_t
        max_sum_log = np.min(-sum_memory_log_t)
        augmented_exp_sum_memory_log_t = np.exp(sum_memory_log_t + max_sum_log)
        for i in range(4):
            p_zt_i[i, t] = augmented_exp_sum_memory_log_t[i] / np.sum(augmented_exp_sum_memory_log_t)
            
    return p_zt_i

def computePijt(memory_log_alpha, memory_log_beta, log_A, multivar, Y_t_data):
    pijt = np.zeros((4,4, T+1))

    for t in range(T):
        sum_memory_log_t = memory_log_alpha[:, t] + memory_log_beta[:, t]

        max_sum_memory_log_t = np.max(sum_memory_log_t)
        log_denominator = max_sum_memory_log_t + np.log(np.sum(np.exp(sum_memory_log_t - max_sum_memory_log_t)))

        tot = 0
        
        for i in range(K):
            for j in range(K):
                log_numerator = memory_log_alpha[:, t][i] + memory_log_beta[:, t+1][j] + log_A[i,j] + np.log(multivar[j].pdf(Y_t_data[t]))
                value = np.exp(log_numerator - log_denominator)
                pijt[i, j, t] = value
                tot+=value
    return pijt

In [410]:
def EM():
    Y_t_data = data['data']
    
    # init
    pi = [0.20, 0.24, 0.25, 0.31]
    centers = [[3.79, -3.64], [ 3.99, -3.64 ], [ -2.03, 4.16 ], [ -3.08, -3.56 ]]

    sigma_0 = np.matrix([[ 0.87, 0.06 ], [ 0.06, 2.21]])
    sigma_1 = np.matrix([[ 0.20, 0.22 ], [ 0.22, 10.40]])
    sigma_2 = np.matrix([[ 2.92, 0.17 ], [ 0.17, 2.77]])
    sigma_3 = np.matrix([[ 6.14, 5.94 ], [ 5.94, 6.07]])

    sigmas = [sigma_0, sigma_1, sigma_2, sigma_3]
    
    multivar = dict()
    for k in range(K):
        m = scipy.stats.multivariate_normal(centers[k], sigmas[k])
        multivar[k] = m
    
    A =  1/10 * np.ones((4,4)) + 6/10 * np.identity(4)
    
    
    p_z0_i = np.zeros((4,1))
    p_zt_i = np.zeros((4, T+1))
    p_zt1_j_zt_i = np.zeros((T+1, T+1))
    
    for j in range(10):
        # E step 
        
        # we use this memory to avoid unnecessary recomputation
        memory_log_alpha = np.zeros((4, T + 1))
        memory_log_beta = np.zeros((4, T + 1))
        log_A = np.log(A)
        log_pi = np.log(pi)
        for i in range(4):
            # we fill memory_log_alpha and memory_log_beta
            _ = computeLogAlpha(i, Y_t_data, log_A, log_pi, memory_log_alpha,multivar, T)
            _ = computeLogBeta(i, Y_t_data, log_A, memory_log_beta,multivar, 0)
        p_zt = computeSmoothing(memory_log_alpha, memory_log_beta)
        pijt = computePijt(memory_log_alpha, memory_log_beta, log_A, multivar, Y_t_data) # todo verify values of pijt
        
        # M step
        
        pi = p_zt[:, 0]
        for i in range(4):
            for j in range(4):
                A[i,j] = np.sum(pijt[i,j,:])/np.sum(p_zt[i, :])
        
        for i in range(4):
            tmp_center_i = [0,0]
            for t in range(T+1):
                tmp_center_i += p_zt[i, t] * Y_t_data[t]
            centers[i] = tmp_center_i / np.sum(p_zt[i, :])
        for i in range(4):  
            tmp_sigma_i = [[0,0],[0,0]]
            for t in range(T+1):
                tmp_mat = np.matrix(Y_t_data[t] - centers[i])
                tmp_sigma_i += p_zt[i, t] * np.dot( np.transpose(tmp_mat),tmp_mat)
            sigmas[i] = tmp_sigma_i / np.sum(p_zt[i, :])
    
    
    return pi, A, centers, sigmas

        
        

In [411]:
EM()

tot: 1.0000000000000715
tot: 1.0000000000004399
tot: 1.0000000000001394
tot: 1.0000000000002889
tot: 1.0000000000004108
tot: 1.0000000000001197
tot: 1.0000000000002467
tot: 1.0000000000000764
tot: 1.000000000000509
tot: 1.0000000000004428


(array([1.25991450e-195, 1.52032706e-114, 2.16867460e-096, 1.00000000e+000]),
 array([[8.83038298e-01, 7.90752830e-02, 4.97884117e-05, 4.62654480e-22],
        [7.50799360e-02, 9.99302279e-01, 9.29261210e-03, 1.96977980e-06],
        [1.89280981e-34, 1.24025654e-07, 9.78859424e-01, 1.94105587e-13],
        [1.51899429e-37, 8.34107864e-03, 1.60758283e-02, 9.62416317e-01]]),
 [array([ 3.64817591, -3.79175116]),
  array([ 4.01998475, -0.23525988]),
  array([-0.14273694,  4.88833015]),
  array([-2.57024544, -2.98918433])],
 [matrix([[ 1.07591866, -0.04164206],
          [-0.04164206,  2.3185532 ]]), matrix([[ 0.28547185, -0.03245054],
          [-0.03245054, 12.17397923]]), matrix([[9.94301637, 3.11731614],
          [3.11731614, 4.30859761]]), matrix([[8.91465714, 8.9443843 ],
          [8.9443843 , 9.46801732]])])

In [366]:
for k in range(4):
    print(k)

0
1
2
3


In [ ]:
# from where you can work